In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-demand-forecasting/meal_info.csv
/kaggle/input/food-demand-forecasting/fulfilment_center_info.csv
/kaggle/input/food-demand-forecasting/train.csv
/kaggle/input/food-demand-forecasting/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

In [4]:
# Loading data
print("Loading data...")
df= pd.read_csv("/kaggle/input/food-demand-forecasting/train.csv")
meal_df = pd.read_csv("/kaggle/input/food-demand-forecasting/meal_info.csv")
center_df = pd.read_csv("/kaggle/input/food-demand-forecasting/fulfilment_center_info.csv")
data= df.merge(center_df,left_on = 'center_id', right_on = 'center_id',how="left")
data= data.merge(meal_df,left_on = 'meal_id', right_on = 'meal_id',how="left")
print(data.head())

Loading data...
        id  week  center_id  meal_id  checkout_price  base_price  \
0  1379560     1         55     1885          136.83      152.29   
1  1466964     1         55     1993          136.83      135.83   
2  1346989     1         55     2539          134.86      135.86   
3  1338232     1         55     2139          339.50      437.53   
4  1448490     1         55     2631          243.50      242.50   

   emailer_for_promotion  homepage_featured  num_orders  city_code  \
0                      0                  0         177        647   
1                      0                  0         270        647   
2                      0                  0         189        647   
3                      0                  0          54        647   
4                      0                  0          40        647   

   region_code center_type  op_area   category cuisine  
0           56      TYPE_C      2.0  Beverages    Thai  
1           56      TYPE_C      2.0  Bev

In [5]:
# Hot encoding
print("Hot encoding...")
def one_hot_encode(features_to_encode, dataset):
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(dataset[features_to_encode])

    encoded_cols = pd.DataFrame(encoder.transform(dataset[features_to_encode]),columns=encoder.get_feature_names())
    dataset = dataset.drop(columns=features_to_encode)
    for cols in encoded_cols.columns:
        dataset[cols] = encoded_cols[cols]
    return dataset

data = data.drop(["id"],axis=1)
features_to_encode = ['meal_id','city_code',"center_id",'center_type', 'category', 'cuisine']
data = one_hot_encode(features_to_encode, data)
y = data["num_orders"]
X= data.drop(["num_orders"],axis = 1)

Hot encoding...


In [6]:
# Train, test partition
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Splitting data...


In [7]:
# Pipeline and fitting
print("Scaling data and creating model...")
RF_pipe = make_pipeline(StandardScaler(), RandomForestRegressor())
RF_pipe.fit(X_train, y_train)

Scaling data and creating model...


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [9]:
# Predictions
print("Predicting result...")
RF_train_y_pred = RF_pipe.predict(X_test)
print(RF_pipe.score(X_test, y_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, RF_train_y_pred)))

Predicting result...
0.8569254459372727
RMSLE: 48.813250440122104


In [13]:
import pickle
os.chdir("/kaggle/working/")
pickle.dump(RF_pipe, open("model.h5", 'wb'))
os.chdir("/kaggle/input/")

In [ ]:
# Second model
print("Creating a second model...")
RF_pipe = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators = 200))
RF_pipe.fit(X_train, y_train)
RF_train_y_pred = RF_pipe.predict(X_test)
print(RF_pipe.score(X_test, y_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, RF_train_y_pred)))